In [2]:
import sklearn
from sklearn.model_selection import train_test_split
import scipy  
import numpy as np
import pandas as pd 
import matplotlib 
from matplotlib import pyplot as plt

# pytorch
import torch
import torch.nn as nn

/Users/soumilchopra/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [10]:
# https://wiki.pathmind.com/neural-network#define
# deep learning counts as a model with 3 or more layers (its not just a buzz word)
# input * weight = guess 
# truth - guess = error
# error * weight's contribution to error = adjustment 
# Essentially: scoring input, calculating loss, and applying an update to the model, 
# then begin that three step process again

# Imagine multiple linear regression is happening at every node of a neural network. 
# For each node of a single layer, input from each node of the previous layer is recombined 
# with input from every other node. That is, the inputs are mixed in different proportions, 
# according to their coefficients, which are different leading into each node of the subsequent
# layer. In this way, a net tests which combination of input is signiicant as it tries to reduce error.

# Once you sum your node inputs to get a prediction on Y value, it is passed through a non-linear function
# Otherwise with each layer y would just increase, which is not what we want
# Nonlinear transforms are usually: tanh, hard tanh etc. (S-shaped functions similar to logistic regression)

# Each weight is just one factor in a deep network that involves many transforms; 
# the signal of the weight passes through activations and sums over several layers, so we use 
# chain rule of calculus to march back through the networks activations and outputs and finally
# arrive at the weight in question, and its relationship to overall error.

# Logistic regression is used for classification rather than regression in the linear sense that most people
# are familiar with. It calculates the probability that a set of inputs match the label

# F(x) = 1/(1 + e^(-x))
# As the input x that triggers a label grows, the expression e to the x shrinks toward zero leaving = 1
# As the input x correlates negatively, the negative gets fliped and e to the x grows larger leaving = 0
# X = sum of the products of all the weights and their corresponding inputs (total signal of NN)
# then we can set a decision threshold to classify as true/false




In [11]:
# Importing and Splitting the data into training and testing sets for both features and labels 
df_normal = pd.read_csv('ptbdb_normal.csv')
df_abnormal = pd.read_csv('ptbdb_abnormal.csv')
df_combined = pd.concat([df_normal, df_abnormal]) # combining both abnormal and normal datasets

df_train, df_test = train_test_split(df_combined, test_size = .2, random_state = 42, shuffle = True)


X_train, X_test = torch.tensor(df_train.drop('188', axis=1).values, dtype=torch.double), torch.tensor(df_test.drop('188', axis=1).values, dtype=torch.double)
Y_train, Y_test = torch.tensor(df_train['188'].values, dtype=torch.double), torch.tensor(df_test['188'].values, dtype=torch.double) 

#display(X_train)
display(Y_train)

tensor([1., 1., 1.,  ..., 1., 0., 1.], dtype=torch.float64)

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() 
        self.flatten = nn.Flatten()
        self.Model = nn.Sequential(
            nn.Linear(1*187, 187, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(187, 187, bias=True),
            nn.ReLU(),
            nn.Linear(187, 60, bias=True),
            nn.ReLU(),
            nn.Linear(60, 60, bias=True),
            nn.ReLU(),
            nn.Linear(60, 30, bias=True),
            nn.ReLU(),
            nn.Linear(30, 10, bias=True),
            nn.ReLU(),
            nn.Linear(10, 1, bias=True),
            nn.ReLU()
            
        )
    
    # To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. 
    # DO NOT CALL model.forward() directly
    def forward(self, input):
        input = self.flatten(input)
        logits = self.Model(input)
        return logits

In [14]:
def train(features, labels, model, loss_eq, optimizer):   
    size = features.size(dim=1) #187 (batch_size)
    for X, Y in zip(features, labels):
        X = X.view(1,187).float()
        Y = Y.view(1,1).float()
        # Compute prediction and loss
        pred = model(X)
        #print(pred)
        loss = loss_eq(pred, Y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if size % 100 == 0:
            loss, current = loss.item(), size * len(X)


def test(features, labels, model, loss_eq):

    num_batches = features.size(dim=0) #Large Number (number of batches)
    total_loss, correct_count = 0, 0

    with torch.no_grad():
        for X, Y in zip(features,labels):
            X = X.view(1,187).float()
            Y = Y.view(1,1).float()
            pred = model(X)
            # Perfomance Metric
            
            # Total Loss
            total_loss += loss_eq(pred, Y).item()
            
            # Number of Times we had the correct prediction
            correct_count += rounded_acc(pred, Y)

    mean_loss = total_loss/num_batches
    accuracy = correct_count/num_batches
    print(f"Accuracy: {(100*accuracy):>0.1f}%, Mean Loss: {mean_loss:>8f} \n")
    
    
def rounded_acc(prediction, actual):
    round_pred = torch.round(prediction).item()
    actual = actual.item()
    
    if round_pred == actual:
        return 1
    return 0

In [15]:
NNModel = NeuralNetwork().to()

# Following Parameters are some of the involved in training of NN
# Learning rate that NN updates model paramaters each epoch 
# Batch size is amount of input data to take before updating model parameters 
# Epochs is repititions of training cycle
learning_rate = 1e-2
epochs = 10
num_layers = 1
num_direction = 1
batch = 187
hidden_size = 1

# Initialize Loss Function and Optimizer
#loss_eq = nn.BCEWithLogitsLoss()
loss_eq = nn.MSELoss()
optimizer = torch.optim.SGD(NNModel.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch: {t+1}")
    train(X_train, Y_train, NNModel, loss_eq, optimizer)
    test(X_test, Y_test, NNModel, loss_eq)
print("Training Complete")
print("Training Error: ")
test(X_train, Y_train, NNModel,loss_eq)

Epoch: 1
Accuracy: 76.3%, Mean Loss: 0.152314 

Epoch: 2
Accuracy: 83.4%, Mean Loss: 0.113034 

Epoch: 3
Accuracy: 85.2%, Mean Loss: 0.103992 

Epoch: 4
Accuracy: 88.3%, Mean Loss: 0.087232 

Epoch: 5
Accuracy: 88.3%, Mean Loss: 0.089488 

Epoch: 6
Accuracy: 89.5%, Mean Loss: 0.081636 

Epoch: 7
Accuracy: 91.3%, Mean Loss: 0.069791 

Epoch: 8
Accuracy: 91.4%, Mean Loss: 0.070190 

Epoch: 9
Accuracy: 91.7%, Mean Loss: 0.067175 

Epoch: 10
Accuracy: 91.8%, Mean Loss: 0.061400 

Training Complete
Training Error: 
Accuracy: 93.4%, Mean Loss: 0.053323 

